# Lab 04: Create Users and Manage Group Membership

This lab will guide you through creating users and managing group memberships using the Microsoft Graph PowerShell SDK.

## Required Permissions

To complete this lab, you need the following Microsoft Graph API permissions:

- **User.ReadWrite.All**: Required to create and manage user accounts
- **Group.ReadWrite.All**: Required to create and manage groups and their memberships

These are privileged permissions that require administrative consent in your Entra ID tenant.

## Authentication with Microsoft Graph

Before you can perform any operations, you need to authenticate to Microsoft Graph using the `Connect-MgGraph` cmdlet. You'll need to sign in with an account that has appropriate administrative privileges.

For this lab, we'll use delegated authentication.

In [ ]:
# Connect to Microsoft Graph with the required permissions
Connect-MgGraph -Scopes "User.ReadWrite.All", "Group.ReadWrite.All"

# Verify the connection and permissions
Get-MgContext

## Task 1: Create Two Users

First, we'll use the `New-MgUser` cmdlet to create two users. We need to set the necessary properties such as `DisplayName`, `UserPrincipalName`, `MailNickname`, and `PasswordProfile`.

In [ ]:
# Create first user
$passwordProfile = @{
    Password = Read-Host -AsSecureString "Enter a secure password"
}

$userParams = @{
    DisplayName = "User One"
    UserPrincipalName = "userone@<YOURDOMAIN>.onmicrosoft.com"
    MailNickName = "userone"
    AccountEnabled = $true
    PasswordProfile = $passwordProfile
}

$user1 = New-MgUser @userParams

# Display the created user
$user1 | Format-List Id, DisplayName, UserPrincipalName

In [ ]:
# Create second user
$passwordProfile = @{
    Password = Read-Host -AsSecureString "Enter a secure password"
}

$userParams = @{
    DisplayName = "User Two"
    UserPrincipalName = "usertwo@<YOURDOMAIN>.onmicrosoft.com"
    MailNickName = "usertwo"
    AccountEnabled = $true
    PasswordProfile = $passwordProfile
}

$user2 = New-MgUser @userParams

# Display the created user
$user2 | Format-List Id, DisplayName, UserPrincipalName

## Task 2: Create a Security Group

Now, we'll use the `New-MgGroup` cmdlet to create a security group and add the first user as a member.

In [ ]:
# Create a security group with the first user as a member
$groupParams = @{
    DisplayName = "SG-Lab04"
    MailEnabled = $false
    SecurityEnabled = $true
    MailNickname = "sg-lab04"
    "Members@odata.bind" = @("https://graph.microsoft.com/v1.0/users/$($user1.Id)")
}
$group = New-MgGroup -BodyParameter $groupParams

# Display the created group
$group | Format-List Id, DisplayName

## Task 3: Add Second User to the Existing Group

Next, we'll use the `New-MgGroupMember` cmdlet to add the second user to the existing group.

In [ ]:
# Add the second user to the group
New-MgGroupMember -GroupId $group.Id -DirectoryObjectId $user2.Id

## Task 4: Verify Group Membership

Finally, we'll verify that both users are members of the group using the `Get-MgGroupMember` cmdlet.

In [ ]:
# Verify group membership using Get-MgGroupMember
Get-MgGroupMember -GroupId $group.Id | Select-Object -ExpandProperty AdditionalProperties

## Challenge Task: Alternative Group Membership Verification

Try to verify the group membership using the `Get-MgGroup` cmdlet.

In [ ]:
# Alternative way to verify group membership using Get-MgGroup
Get-MgGroup -GroupId $group.Id -Expand members | Select-Object -ExpandProperty members |
Select-Object -ExpandProperty AdditionalProperties

### Explanation of the Get-MgGroup Query

The above PowerShell command demonstrates an alternative approach to retrieving group members using the `Get-MgGroup` cmdlet instead of the dedicated `Get-MgGroupMember` cmdlet we used earlier.

Here's how it works:

1. `Get-MgGroup -GroupId $group.Id` retrieves the group object by its ID (stored in the `$group.Id` variable).

2. The `-Expand members` parameter is a crucial part of this command. It uses the OData $expand query parameter to include related entities (in this case, the group members) directly in the response, rather than requiring a separate API call to retrieve them.

3. `Select-Object -ExpandProperty members` extracts just the members collection from the group object. Without this, we would get the entire group object with the members as just one property.

4. Finally, `Select-Object -ExpandProperty AdditionalProperties` extracts the properties of each member object. In the Microsoft Graph PowerShell SDK, many object properties are stored in the AdditionalProperties property as a generic dictionary. **Note:** When working with generic dictionaries, property names are case-sensitive, so ensure you use the correct casing when accessing specific properties.

This approach demonstrates how you can use OData query parameters like $expand directly through the Microsoft Graph PowerShell SDK to optimize your API calls by retrieving related data in a single operation.

## Conclusion

You have successfully completed the following tasks:

1. Created two users in Entra ID
2. Created a security group with one initial member
3. Added another user to the group
4. Verified the group membership using different methods

These skills are essential for managing users and groups programmatically using Microsoft Graph PowerShell.